## 🤖 4. ML Pipeline with Vertex AI & Feast
 In this notebook, we:
 - Use Feast to fetch features for training
 - Train a regression model using scikit-learn
 - Log metrics and save model to GCS
 - Deploy model to Vertex AI

### ⚙️ Setup & Imports

In [ ]:
# =================== ✅ CODE CELL ===================
!pip install feast google-cloud-storage scikit-learn joblib --quiet

import os
import joblib
import pandas as pd
from feast import FeatureStore
from google.cloud import storage
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import json
import numpy as np

### 🧠 Step 1: Retrieve Features from Feast

In [ ]:
store = FeatureStore(repo_path="my_feature_repo")

entity_df = pd.DataFrame({
    "user_id": list(range(1, 51)),
    "event_timestamp": pd.to_datetime("now")
})

training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "user_features:avg_rating",
        "user_features:num_ratings"
    ],
).to_df()

print("✅ Retrieved training features")
training_df.head()

### 🏋️ Step 2: Train Model

In [ ]:
X = training_df[["avg_rating", "num_ratings"]]
y = training_df["avg_rating"]  # For demo; replace with correct label in real use

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

y_pred = model.predict(X)
rmse = np.sqrt(mean_squared_error(y, y_pred))
print(f"✅ RMSE: {rmse:.4f}")

### 💾 Step 3: Save Model and Metrics

In [ ]:
output_dir = os.environ.get("AIP_MODEL_DIR", "/tmp/model")
os.makedirs(output_dir, exist_ok=True)

model_path = os.path.join(output_dir, "model.joblib")
joblib.dump(model, model_path)

metrics_path = os.path.join(output_dir, "metrics.json")
with open(metrics_path, "w") as f:
    json.dump({"rmse": float(rmse)}, f)

print(f"✅ Saved model to {model_path}")
print(f"✅ Saved metrics to {metrics_path}")

### ☁️ Step 4: Upload to GCS (Optional)

In [ ]:
bucket_name = "your-model-artifacts-bucket"
destination_blob = "models/recommender/model.joblib"

client = storage.Client()
bucket = client.bucket(bucket_name)
blob = bucket.blob(destination_blob)
blob.upload_from_filename(model_path)

print(f"✅ Model uploaded to gs://{bucket_name}/{destination_blob}")

### 🚀 Step 5: Deploy to Vertex AI

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project="your-project-id", location="us-central1")

model = aiplatform.Model.upload(
    display_name="recommender-regression-model",
    artifact_uri=f"gs://{bucket_name}/models/recommender/",
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest",
)

print("✅ Model uploaded to Vertex AI")

## ✅ Summary
 - Fetched training data from Feast
 - Trained and evaluated regression model
 - Saved and uploaded model artifacts to GCS
 - Deployed model to Vertex AI
